# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, like the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.

If you need to start again, go to Kernel menu >> Restart kernel.

In [11]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. Also, double check you have the right kind of API token with the right permissions. You should find it on [this webpage](https://platform.openai.com/api-keys) and it should show with Permissions of "All". If not, try creating another key by:
- Pressing "Create new secret key" on the top right
- Select **Owned by:** you, **Project:** Default project, **Permissions:** All
- Click Create secret key, and use that new key in the code and the `.env` file (it might take a few minutes to activate)
- Do a Kernel >> Restart kernel, and execute the cells in this Jupyter lab starting at the top
4. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
5. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [12]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY','your-key-if-not-using-env')
openai = OpenAI()

In [13]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [14]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [15]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. Return results in Vietnamese\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [16]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [17]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [18]:
summarize("https://edwarddonner.com")

'# Tóm tắt trang web của Edward Donner\n\nTrang web này thuộc về Edward Donner, một lập trình viên và người thử nghiệm với các mô hình ngôn ngữ lớn (LLMs). Ông là đồng sáng lập và Giám đốc công nghệ của Nebula.io, tập trung vào việc áp dụng trí tuệ nhân tạo để giúp mọi người khám phá tiềm năng của họ và quản lý nhân tài.\n\n## Tin tức và thông báo\n- **21 tháng 12, 2024**: Chào mừng các nhà khoa học dữ liệu SuperData!\n- **13 tháng 11, 2024**: Tài nguyên về việc làm chủ AI và Kỹ thuật LLM.\n- **16 tháng 10, 2024**: Tài nguyên từ Kỹ sư phần mềm đến Nhà khoa học dữ liệu AI.\n- **6 tháng 8, 2024**: "Outsmart LLM Arena" - một đấu trường cho các LLM cạnh tranh nhau trong trận chiến "ngoại giao và mưu mẹo".\n\nEdward thích viết mã, sản xuất nhạc điện tử nghiệp dư và theo dõi các xu hướng công nghệ trên Hacker News.'

In [19]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [20]:
display_summary("https://edwarddonner.com")

# Tóm tắt trang web - Edward Donner

Trang web của Edward Donner chủ yếu tập trung vào việc chia sẻ kiến thức và kinh nghiệm của anh trong lĩnh vực trí tuệ nhân tạo (AI) và các mô hình ngôn ngữ lớn (LLMs). Trong phần giới thiệu, Edward chia sẻ sở thích của mình với lập trình, sản xuất nhạc điện tử và tư duy về công nghệ. Anh là đồng sáng lập và giám đốc công nghệ của Nebula.io, nơi ứng dụng AI để giúp mọi người khám phá tiềm năng của họ.

## Tin tức và thông báo
- **21 tháng 12 năm 2024**: Chào mừng các SuperDataScientists!
- **13 tháng 11 năm 2024**: Tài nguyên để thành thạo Kỹ thuật AI và LLM.
- **16 tháng 10 năm 2024**: Tài nguyên chuyển đổi từ Kỹ sư Phần mềm sang Kỹ sư Dữ liệu AI.
- **6 tháng 8 năm 2024**: Arena Outsmart LLM – một cuộc chiến về ngoại giao và sự xảo trá.

In [21]:
display_summary("https://cnn.com")

# Tóm tắt về trang web CNN

CNN là một trong những nguồn tin tức hàng đầu thế giới, cung cấp tin tức nóng hổi và video về nhiều chủ đề phong phú, bao gồm chính trị, kinh doanh, sức khỏe, thể thao và giải trí. Ngoài ra, CNN cũng theo dõi các sự kiện toàn cầu quan trọng như cuộc chiến Ukraine-Russia và xung đột Israel-Hamas.

## Tin tức nổi bật
- **Cuộc chiến Israel-Hamas**: Đã có thông báo về một thỏa thuận ngừng bắn và việc trao đổi con tin, với Hamas dự kiến sẽ thả 33 con tin và Israel đồng ý thả hàng trăm tù nhân Palestine.
- **Hoả hoạn ở Los Angeles**: Một làn sóng hoả hoạn nghiêm trọng đã xảy ra, tuy nhiên, mối đe dọa chính từ các đám cháy đã được giảm thiểu.

CNN cũng cung cấp văn bản và video về các chủ đề như công nghệ, sức khỏe, khoa học, và môi trường, nhằm phục vụ độc giả trên toàn cầu.

In [22]:
display_summary("https://anthropic.com")

# Tóm tắt trang web Anthropic

Trang web **Anthropic** tập trung vào nghiên cứu và phát triển AI, với trọng tâm đặc biệt vào sự an toàn của trí tuệ nhân tạo. Họ giới thiệu mô hình AI mới nhất của mình, **Claude 3.5 Sonnet**, và các sản phẩm như **Claude cho Doanh Nghiệp**.

## Thông báo nổi bật
- **Ra mắt Claude 3.5 Sonnet** - Mô hình AI thông minh nhất của họ, đã có sẵn để sử dụng.
- **Giới thiệu mô hình Claude 3.5 Haiku** - Cập nhật mới cho sản phẩm của họ.

## Nghiên cứu và An toàn AI
Anthropic sử dụng một đội ngũ đa ngành để tạo ra các hệ thống AI đáng tin cậy và có lợi, với nhiều nghiên cứu về an toàn AI và phản hồi từ AI để phát triển trí tuệ nhân tạo một cách an toàn và hiệu quả.

In [23]:
display_summary("https://kfsp.vn")

# Tóm tắt trang web Kungfu Stocks Pro

Trang web Kungfu Stocks Pro cung cấp thông tin và công cụ liên quan đến thị trường chứng khoán. Nội dung chính bao gồm các phân tích về cổ phiếu, tín hiệu giao dịch và các dự báo thị trường.

## Tin tức và thông báo

Hiện tại, không có thông tin cụ thể về các tin tức hoặc thông báo trên trang web do nội dung của trang yêu cầu bật JavaScript để hiển thị.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. Please push your code afterwards so I can share it with other students!

In [24]:
display_summary("https://openai.com")

Xin lỗi, nhưng tôi không thể truy cập nội dung cụ thể của trang web mà bạn đã đề cập, vì tôi chỉ có khả năng phân tích nội dung văn bản mà bạn cung cấp. Tuy nhiên, nếu bạn cung cấp nội dung cụ thể từ trang web đó, tôi sẽ giúp bạn tóm tắt bằng tiếng Việt.

In [25]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = '..\\path\\to\\chromedriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

ModuleNotFoundError: No module named 'selenium'

In [ ]:
display_summary("https://openai.com")

In [ ]:
display_summary("https://edwarddonner.com")

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://kfsp.vn")